In [1]:
import numpy as np
import pandas as pd

In [2]:
# df_datubasea = pd.read_csv(r'G:\My Drive\Uni\NLP Master\Praktika Euskara Dialektoak\Datu-basea_ euskalkien ezaugarri adierazgarriak eta bereizgarriak\ESB-datu-basea-zutabeka-24XI-18.csv', sep='\t')
# datubasea = df_datubasea.to_numpy()
# arrays = []
# for i in range(649):
#     arrays.append(datubasea[145*i:145*(i+1),3])
# arrays = np.array(arrays)
# matrix = np.column_stack(arrays)
# df = pd.DataFrame(matrix.T)
# df.to_csv("file.csv")

columnas = numero de items

filas = ((n*n)-n)/2

In [12]:
distances = pd.read_csv('./Datu-basea_ euskalkien ezaugarri adierazgarriak eta bereizgarriak/Matriz-distancias del ESB-Diatech z-24-04-02.csv').drop("Unnamed: 0", axis=1).to_numpy()
clusters = pd.read_csv('./Datu-basea_ euskalkien ezaugarri adierazgarriak eta bereizgarriak/fanny_clusters.csv').T.to_numpy()[0]-1
membership = pd.read_csv('./Datu-basea_ euskalkien ezaugarri adierazgarriak eta bereizgarriak/fanny_membership.csv').to_numpy()
datubasea = pd.read_csv('./Datu-basea_ euskalkien ezaugarri adierazgarriak eta bereizgarriak/ESB-datu-basea-taula.csv').T.drop('Unnamed: 0').T.to_numpy()

- Datos de entrada: 
    - Los datos 
    - n_clusters 
    - Distancia (IRI, Levenshtein, ...)

- Lo que tenemos que hacer: 

    - Calcular la distancia
    - Aplicar la clasificacion no determinista.
        - Interna: u_ij (Probabilidades de pertenencia)
        - Externa: Grupo 1: p_i, .... (El grupo que asignas es la mayor probabilidad)
    - Matriz de interdistancias para cada item
        - Estabilidad, Variabilidad y Diferenciacion
        - Listado de items que son los mas distintivos y estables

- Salida:
    - Poblaciones de cada grupo
    - Los u_ij
    - Listado de los items (top 10 adibidez)


- Ver como está implementado Diatech (Gotzon-i eskau)
- Ver la distancia Levenshtein si esta implementado
 

In [5]:
membership

array([[0.33333338, 0.33333328, 0.33333334],
       [0.33333343, 0.33333323, 0.33333334],
       [0.33333331, 0.33333336, 0.33333333],
       [0.33333344, 0.33333321, 0.33333335],
       [0.33333337, 0.3333333 , 0.33333334],
       [0.33333341, 0.33333325, 0.33333334],
       [0.3333334 , 0.33333325, 0.33333334],
       [0.33333331, 0.33333336, 0.33333333],
       [0.33333332, 0.33333335, 0.33333333],
       [0.33333338, 0.33333329, 0.33333334],
       [0.33333324, 0.33333343, 0.33333332],
       [0.33333326, 0.33333342, 0.33333332],
       [0.33333333, 0.33333334, 0.33333333],
       [0.33333343, 0.33333323, 0.33333334],
       [0.33333344, 0.33333322, 0.33333335],
       [0.33333344, 0.33333321, 0.33333335],
       [0.33333342, 0.33333324, 0.33333334],
       [0.33333325, 0.33333343, 0.33333332],
       [0.33333324, 0.33333344, 0.33333332],
       [0.33333324, 0.33333344, 0.33333332],
       [0.33333343, 0.33333323, 0.33333334],
       [0.33333331, 0.33333336, 0.33333333],
       [0.

In [ ]:
int(145*(145-1)/2)
print("a")

10440

In [12]:
a = np.zeros((int(145*(145-1)/2),649))
np.savetxt('scores.csv', a)

In [6]:
def bilbao_d(items1, items2):

    bat = sum(1 for item in items1 if item in items2) 
    bat += sum(1 for item in items2 if item in items1)
    
    return 1 - (bat/(len(items1)+len(items2)))

In [10]:
bilbao_d(["a", "b"], ["c"])

1.0

In [ ]:
def kalkulatu_estabilitatea(datubasea, membership, galdera, cluster):
    
    c, elementuak = cluster
    n = len(elementuak)
    
    batura = 0
    pisuen_batura = 0
    for i in range(n):
        for j in range(i+1, n):
            if datubasea[galdera, i] != datubasea[galdera, i] or datubasea[galdera, j] != datubasea[galdera, j]:
                distantzia = 1
            else: 
                distantzia = bilbao_d(datubasea[galdera, i].split(','), datubasea[galdera, j].split(','))
                
            pisuak = membership[i, c] * membership[j, c]
            pisuen_batura += pisuak
            batura += distantzia * pisuak

    return batura*2/(pisuen_batura)

def kalkulatu_bariabilitatea(datubasea, membership, galdera, cluster1, cluster2):
    
    c1, elementuak1 = cluster1
    n1 = len(elementuak1)

    c2, elementuak2 = cluster2
    n2 = len(elementuak2)

    batura = 0
    pisuen_batura = 0
    for i in range(n1):
        for j in range(n2):
            if datubasea[galdera, i]!=datubasea[galdera, i] or datubasea[galdera, j]!=datubasea[galdera, j]:
                distantzia = 1
            else:
                distantzia = bilbao_d(datubasea[galdera, i].split(','), datubasea[galdera, j].split(','))
            pisuak = membership[i, c1] * membership[j, c2]
            pisuen_batura += pisuak
            batura += distantzia * pisuak

    return batura/(pisuen_batura)

def kalkulatu_diferentziazioa(datubasea, membership, galdera, cluster1, cluster2):
    
    estabilitatea1 = kalkulatu_estabilitatea(datubasea, membership, galdera, cluster1)
    estabilitatea2 = kalkulatu_estabilitatea(datubasea, membership, galdera, cluster2)
    bariabilitatea1_2 = kalkulatu_bariabilitatea(datubasea, membership, galdera, cluster1, cluster2)
    
    return bariabilitatea1_2 / max(estabilitatea1, estabilitatea2, 1e-6)


In [14]:
c1 = 0
c2 = 2

cluster1 = (c1, np.where(clusters == c1)[0])
cluster2 = (c2, np.where(clusters == c2)[0])

diferenetziazioak = {}
for galdera in range(len(datubasea)):
    diferenetziazioak[galdera] = kalkulatu_diferentziazioa(datubasea, membership, galdera, cluster1, cluster2)

sorted_dict = dict(sorted(diferenetziazioak.items(), key=lambda item: item[1], reverse=True))

top3_item = list(sorted_dict.keys())[:3]

In [18]:
cluster1

(0, array([  0,   4,   9,  45,  54,  58,  63,  98, 107, 126, 127, 141, 144]))

In [ ]:
print(datubasea[top3_item[0], cluster1[1]])
print(datubasea[top3_item[0], cluster2[1]])
print("------")
print(datubasea[top3_item[1], cluster1[1]])
print(datubasea[top3_item[1], cluster2[1]])
print("------")
print(datubasea[top3_item[2], cluster1[1]])
print(datubasea[top3_item[2], cluster2[1]])

['dakoz' 'dakoz' 'dakoz' 'zakoz' 'zaizko' 'zaizka,zaizkio,zaizko' 'zazkio'
 'zaizkio' 'zakote' 'zazko' 'zazko' 'zaitzo' 'zaie,zaitzo']
['jakoz' 'jakoz' 'jakoz' 'jakoz' 'jakoz' 'zaizko' 'zaizkio,zaizka' 'zaio'
 'zaizkio' 'zazkio' 'zazkio' 'zaizko' 'dakote' 'zaizko' 'zei']
------
['dako' 'dako' 'dako' 'zako,zako' 'zaio' 'zaio' 'zaio' 'zaio' 'zako,zaio'
 'zako' 'zako' 'zaio,zao' 'zako,zao,zaio']
['jako' 'jako' 'jako' 'jako' 'jako' 'zako' 'zaio' 'zaio' 'zaio' 'zaio'
 'zaio' 'zaio' 'dako,zako' 'zako' 'zako']
------
['dat' 'dat' 'dat' 'zat' 'zat' 'zat' 'zait' 'zait' 'zaut' 'zaut' 'zait'
 'zait' 'zait']
['jat' 'jat' 'jat' 'jat' 'jat' 'Zait,zat' 'zait,zat' 'zat' 'zait'
 'zat,zait' 'zait' 'zait' 'zaut' 'zaut' 'zaut,zat']
